In [2]:
import pickle
import boto3
import uuid
import botocore
from botocore.exceptions import ClientError
import os, time, json, time
from datetime import datetime

from misc import load_from_yaml, save_to_yaml
# import s3, iam, lftn, glue, lambdafn as lfn, sns, eventbridge as event
# from lambdafn import build_lambda_package, print_latest_lambda_logs

from dotenv import load_dotenv
load_dotenv(os.getenv("MY_AWS_DIR", "") + "/.env")

from aws_log_extractor import LogManager, LogFilter, LogLevel

from mylogger import CustomLogger
logger = CustomLogger()

In [3]:
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')
# VPC_ID            = os.environ['AWS_DEFAULT_VPC']
# SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
# SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
# SUBNET_ID         = SUBNET_IDS[0]

logger.info(f"VPC_ID: {ACCOUNT_ID}")

INFO: 2025-09-16 10:43:55 [2366322474.py:8] VPC_ID: 530976901147


In [4]:
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
iam_client           = boto3.client('iam')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)
dynamodb_client      = boto3.client('dynamodb')
events_client        = boto3.client('events')
lambda_client        = boto3.client('lambda')
sns_client           = boto3.client('sns')
cw_logs_client       = boto3.client('logs')
logs_client = boto3.client("logs")

# Create a CloudWatch client for Logs
logs_client = boto3.client('logs')

#### S3 Bucket Notification Test

In [5]:
# Your bucket name
bucket_name = "datalake-bkt-06012025"  # replace with var.datalake_bkt.bucket
sales_file_sufix = datetime.now().strftime("%Y%m%d%H%M%S")
sales_file_key = f"bronze/sales/sales_{sales_file_sufix}.csv"

In [5]:

# # Put a test object (this should trigger the Lambda)
# response = s3_client.put_object(
#     Bucket=bucket_name, Key=test_key, Body=json.dumps({"test": "notification"})
# )

# Put a test object (this should trigger the Lambda)
response = s3_client.upload_file(
    Bucket=bucket_name, 
    Key=sales_file_key,
    Filename="/Users/am/mydocs/Software_Development/DEDS/aws/glue/data/sales1.csv"
)

print("PutObject response:")
print(response)


PutObject response:
None


In [ ]:
response = s3_client.delete_objects(
    Bucket=bucket_name,
    Delete={
        "Objects": [
            {
                "Key": sales_file_key,
            },
        ],
    },
)

In [7]:
[
    # s3_client.delete_object(Bucket=bucket_name, Key=item["Key"])
    print(item["Key"])
    for item in s3_client.list_objects_v2(Bucket="datalake-bkt-06012025", Prefix="DQE")[
        "Contents"
    ]
    if item["Key"]
]

DQE-Results/customers/
DQE-Results/customers/run-customers_dqe_result_node1758035183774-1-part-block-0-r-00000-snappy.parquet
DQE-Results/customers/run-customers_dqe_result_node1758035183774-1-part-block-0-r-00001-snappy.parquet
DQE-Results/customers/run-customers_dqe_result_node1758035183774-1-part-block-0-r-00002-snappy.parquet
DQE-Results/customers/run-customers_dqe_result_node1758035183774-1-part-block-0-r-00003-snappy.parquet
DQE-Results/customers/run-customers_dqe_result_node1758035183774-1-part-block-0-r-00004-snappy.parquet
DQE-Results/customers_failed/
DQE-Results/sales/


[None, None, None, None, None, None, None, None]